INSTALLS

In [1]:
!pip install ijson
!pip install shap
!pip install pdpbox

                                              0.0/48.2 kB ? eta -:--:--
     -----------------                        20.5/48.2 kB ? eta -:--:--
     ------------------------               30.7/48.2 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 48.2/48.2 kB 486.3 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\Trelans\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


JSON TO CSV CONVERSION AGE FILTERING


In [ ]:
import pandas as pd

ads_non_duplicate_path = '/content/drive/MyDrive/Bitirme/ads_non_duplicate.json'
ads_dataframe = '/content/drive/MyDrive/Bitirme/ads_dataframe.csv'

with open(ads_non_duplicate_path, 'r') as file:
    data = json.load(file)

processed_data = []

for item in data:
   
    selected_attributes = {
        'ad_creation_time': item.get('ad_creation_time'),
        'ad_creative_bodies': item.get('ad_creative_bodies'),
        'currency': item.get('currency'),
        'impressions': item.get('impressions'),
        'spend': item.get('spend'),
        'delivery_by_region': item.get('delivery_by_region', []),
        'demographic_distribution': item.get('demographic_distribution', [])
    }
    processed_data.append(selected_attributes)

df = pd.DataFrame(processed_data)

df.to_csv(ads_dataframe, index=False)

IMPORTS

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import ijson
from sklearn.feature_extraction.text import TfidfVectorizer
import shap
from pdpbox import pdp
import joblib
import json
import gzip
import pdpbox
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import ndcg_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from google.colab import drive
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import PowerTransformer
import nltk
import ast
from nltk.corpus import stopwords
drive.mount('/content/drive')


ads_dataset_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads.json'
ads_downsized_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_downsized.json'
ads_non_duplicate_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_non_duplicate.json'
ads_dataframe = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_dataframe.csv'
ads_non_duplicate_path = '/content/drive/MyDrive/CS491MLMODEL/us/us/ads_non_duplicate.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


CONVERT INTO DATAFRAME AGE FILTERING


In [ ]:
import pandas as pd
import ast

# Assuming 'df' is your original DataFrame

# Create a new DataFrame
new_df = pd.DataFrame()

# Convert 'ad_creation_time' to datetime
new_df['ad_creation_time'] = pd.to_datetime(df['ad_creation_time'])

# Extract 'ad_creative_bodies'
new_df['ad_creative_bodies'] = df['ad_creative_bodies'].apply(lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else x).astype(str)

# Extract 'lower_bound' and 'upper_bound' values for impressions and spend
new_df['impressions_lower'] = df['impressions'].apply(lambda x: ast.literal_eval(x).get('lower_bound', None)).astype("float")
new_df['impressions_upper'] = df['impressions'].apply(lambda x: ast.literal_eval(x).get('upper_bound', None)).astype("float")
new_df['spend_lower'] = df['spend'].apply(lambda x: ast.literal_eval(x).get('lower_bound', None)).astype("float")
new_df['spend_upper'] = df['spend'].apply(lambda x: ast.literal_eval(x).get('upper_bound', None)).astype("float")

# Create columns for unique region names
'''
unique_regions = set()
for item in df['delivery_by_region']:
    try:
        regions = ast.literal_eval(item)
        for region in regions:
            if 'region' in region:
                unique_regions.add(region['region'])
    except (SyntaxError, ValueError):
        pass

for region in unique_regions:
    new_df[f'percentage_{region}'] = None
'''
# Create columns for age intervals
unique_age_intervals = set()
for item in df['demographic_distribution']:
    try:
        demographics = ast.literal_eval(item)
        for demographic in demographics:
            age = demographic.get('age', None)
            gender = demographic.get('gender', None)
            if age is not None and gender is not None:
                age_gender_key = f'{age}_{gender}'
                unique_age_intervals.add(age_gender_key)
    except (SyntaxError, ValueError):
        pass

for age_interval in unique_age_intervals:
    new_df[f'percentage_{age_interval}'] = None
'''
# Iterate through the data to fill in information or use percentages as empty
for i, item in enumerate(df['delivery_by_region']):
    try:
        regions = ast.literal_eval(item)
        for region in regions:
            if 'region' in region:
                new_df.at[i, f'percentage_{region["region"]}'] = region.get('percentage', None)
    except (SyntaxError, ValueError):
        pass
'''
for i, item in enumerate(df['demographic_distribution']):
    try:
        demographics = ast.literal_eval(item)
        for demographic in demographics:
            age = demographic.get('age', None)
            gender = demographic.get('gender', None)
            if age is not None and gender is not None:
                age_gender_key = f'{age}_{gender}'
                new_df.at[i, f'percentage_{age_gender_key}'] = demographic.get('percentage', None)
    except (SyntaxError, ValueError):
        pass

# Display data types and the head of the new DataFrame
print(new_df.dtypes)
print(new_df.head())


DATA DOWNSIZING

In [ ]:
json_item_heuristic = 3000000

def downsize_data(num_of_items, input_dataset_path, output_dataset_path):
    counter = 0
    with open(input_dataset_path, 'r') as input_file:
        parser = ijson.items(input_file, 'item')

        with open(output_dataset_path, 'a') as outfile:  # Open output file once
            outfile.write('[\n')
            # Use tqdm to create a progress bar
            for item in tqdm(parser, total=int(num_of_items), unit=" items"):
                if counter > num_of_items:
                    break
                counter += 1
                json.dump(item, outfile)
                if counter > num_of_items:
                  outfile.write('\n')  # Add a newline between items
                else:
                  outfile.write(',\n')  # Add a newline between items

            outfile.write(']\n')
            outfile.close()

downsize_data(json_item_heuristic, ads_dataset_path, ads_downsized_path)

767264 items [05:44, 2227.86 items/s]                        


DATA DUPLICATE REMOVAL

In [ ]:
def remove_duplicates(filename, column, output_filename):
    json_item_heuristic = 3000
    first_item = True
    removed_content = 0
    unique_values = set()

    with open(filename, 'rb') as file:
        items = ijson.items(file, 'item')

        with open(output_filename, 'w') as outfile:
            outfile.write('[\n')
            for item in tqdm(items, total=int(json_item_heuristic), unit=" items"):
                if 'ad_creative_bodies' in item:
                    column_value = tuple(item['ad_creative_bodies'])

                    if (column_value not in unique_values) and (len(column_value) == 1):
                        unique_values.add(column_value)
                        if first_item:
                            json.dump(item, outfile)
                            first_item = False
                        else:
                            outfile.write(',\n')
                            json.dump(item, outfile)
                    else:
                        removed_content += 1
                else:
                    removed_content += 1
                    pass
            outfile.write(']\n')
    print(f"Amount of ads that have been filtered: {removed_content}")


remove_duplicates(ads_downsized_path, 'a', ads_non_duplicate_path)

JSON TO CSV CONVERSION

In [ ]:
with open(ads_non_duplicate_path, 'r') as file:
    data = json.load(file)

processed_data = []
for item in data:
    selected_attributes = {'ad_creation_time': item['ad_creation_time'],
                           'ad_creative_bodies': item['ad_creative_bodies'],
                           'currency': item['currency'],
                           'impressions': item['impressions'],
                           'spend': item['spend']}
    processed_data.append(selected_attributes)

df = pd.DataFrame(processed_data)

df.to_csv(ads_dataframe, index=False)

LOAD DATAFRAME & GET INFORMATION

In [ ]:
df = pd.read_csv(ads_dataframe)
print(df.dtypes)
df.head()

CONVERT INTO DATAFRAME

In [ ]:
new_df = pd.DataFrame()

new_df['ad_creation_time'] = pd.to_datetime(df['ad_creation_time'])
new_df['ad_creative_bodies'] = df['ad_creative_bodies'].apply(lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else x).astype(str)


# Extract 'lower_bound' and 'upper_bound' values if they exist, otherwise use NaN
new_df['impressions_lower'] = df['impressions'].apply(lambda x: ast.literal_eval(x).get('lower_bound', None)).astype("float")
new_df['impressions_upper'] = df['impressions'].apply(lambda x: ast.literal_eval(x).get('upper_bound', None)).astype("float")

new_df['spend_lower'] = df['spend'].apply(lambda x: ast.literal_eval(x).get('lower_bound', None)).astype("float")
new_df['spend_upper'] = df['spend'].apply(lambda x: ast.literal_eval(x).get('upper_bound', None)).astype("float")

print(new_df.dtypes)

new_df.head()

MERGE COLUMNS AND SORT ACCORDING TO DATE

In [ ]:
ads_dataframe = pd.DataFrame()
ads_dataframe['ad_creation_time'] = new_df['ad_creation_time']
ads_dataframe['ad_creative_bodies'] = new_df['ad_creative_bodies']
ads_dataframe['cpi'] = (
    (new_df['spend_lower'].fillna(new_df['spend_upper']) / 2 +
     new_df['spend_upper'].fillna(new_df['spend_lower']) / 2) /
    (new_df['impressions_lower'].fillna(new_df['impressions_upper']) / 2 +
     new_df['impressions_upper'].fillna(new_df['impressions_lower']) / 2)
)

ads_dataframe = ads_dataframe.sort_values(by='ad_creation_time')
ads_dataframe

VECTORIZE TEXTUAL CONTENT

In [ ]:
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

def generate_doc_vectors(docs, model, num_features):
    doc_vectors = [average_word_vectors(doc, model, model.wv.index_to_key, num_features) for doc in docs]
    return np.array(doc_vectors)

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Determine the split index
split_index = int(0.8 * len(ads_dataframe))

train_data = ads_dataframe.iloc[:split_index]
test_data = ads_dataframe.iloc[split_index:]

X_train = train_data['ad_creative_bodies'].astype(str)
y_train = train_data['cpi']

X_test = test_data['ad_creative_bodies'].astype(str)
y_test = test_data['cpi']

# Tokenization and stopword removal
tokenized_text_train = X_train.apply(tokenize_and_remove_stopwords)
tokenized_text_test = X_test.apply(tokenize_and_remove_stopwords)

# Word2Vec model training
word2vec_model = Word2Vec(sentences=tokenized_text_train, vector_size=100, window=5, min_count=1, workers=4)

# Generate document vectors
X_train_word2vec = generate_doc_vectors(tokenized_text_train, word2vec_model, 100)
X_test_word2vec = generate_doc_vectors(tokenized_text_test, word2vec_model, 100)

#word2vec_model.save("word2vec_model.model")

TRAIN ML MODEL

In [ ]:
# Model training and evaluation
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor.fit(X_train_word2vec, y_train)

SAVE MODEL

In [ ]:
joblib.dump(gb_regressor, '/content/drive/MyDrive/CS491MLMODEL/us/us/gradient_boosting_model.pkl')
#loaded_model = joblib.load('/content/drive/MyDrive/CS491MLMODEL/us/us/gradient_boosting_model.pkl')

LOAD MODEL

In [ ]:
loaded_model = joblib.load('/content/drive/MyDrive/CS491MLMODEL/us/us/gradient_boosting_model.pkl')

TEST WITH NDCG

In [ ]:
# Function to create subsets of data
def create_data_subsets(data, group_size):
    grouped_data = [data.iloc[i : i + group_size] for i in range(0 , data.shape[0], group_size)]
    return grouped_data

def calculate_ndcg_for_groups(groups, model):
    ndcg_scores = []
    for group in tqdm(groups, total=len(groups), unit=" items"):
        tokenized_X_train_group = group['ad_creative_bodies'].apply(tokenize_and_remove_stopwords)
        X_train_word2vec_group = generate_doc_vectors(tokenized_X_train_group, word2vec_model, 100)

        y_group_pred = model.predict(X_train_word2vec_group)

        # Assuming lower CPI values are more relevant
        relevance_scores = 1 / y_group_pred

        # Normalize relevance scores (optional)
        #relevance_scores = (relevance_scores - np.min(relevance_scores)) / (np.max(relevance_scores) - np.min(relevance_scores))

        y_group_true = 1 / group['cpi'].values

        try:
            ndcg = ndcg_score([y_group_true], [relevance_scores])
            ndcg_scores.append(ndcg)
        except Exception as e:
            print(y_group_true)
            print(relevance_scores)

    return ndcg_scores

group_size = 4
test_data_subsets = create_data_subsets(test_data, group_size)

ndcg_scores = calculate_ndcg_for_groups(test_data_subsets, gb_regressor)

PRINT NDCG SCORES

In [ ]:
# Combine NDCG scores across groups (simple average in this example)
average_ndcg_across_groups = sum(ndcg_scores) / len(ndcg_scores)

print(f"Average NDCG Across Groups: {round(average_ndcg_across_groups,2)}")

RECOMMENDATION MODEL TRAINING

In [ ]:
# Determine the split index
split_index = int(0.8 * len(ads_dataframe))

train_data = ads_dataframe.iloc[:split_index]
test_data = ads_dataframe.iloc[split_index:]

# Vectorize the text data using TF-IDF
max_features = 50  # Choose the top most important words
vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
X_train_suggestion = vectorizer.fit_transform(train_data['ad_creative_bodies'])
X_test_suggestion = vectorizer.transform(test_data['ad_creative_bodies'])

# Train a regression model
gb_regressor_suggestion = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor_suggestion.fit(X_train_suggestion, train_data['cpi'])

PARTIAL DEPENDENCE PLOT

In [ ]:
# Choose a feature for the Partial Dependence Plot (e.g., the first feature)
feature_index = 1
feature_name = vectorizer.get_feature_names_out()[feature_index]

# Create the Partial Dependence Plot.
pdp_plotter = pdpbox.pdp.PDPIsolate(gb_regressor_suggestion, pd.DataFrame.sparse.from_spmatrix(X_test_suggestion, columns = vectorizer.get_feature_names_out()), vectorizer.get_feature_names_out(), feature_name, "zattiri-zort", n_classes=0)
pdp_plot = pdp_plotter.plot()
pdp_plot[0]

SHAP VALUE

In [ ]:
import matplotlib.pyplot as plt
import shap

def shap_value_plot(shap_values_instance):
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.bar(feature_names, shap_values_instance[0], color='lightblue')
    plt.title('SHAP values for instance index {}'.format(instance_index))
    plt.xlabel('Feature')
    plt.ylabel('SHAP value')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

feature_names = vectorizer.get_feature_names_out()

explainer = shap.TreeExplainer(gb_regressor_suggestion)

instance_index = 166  # Index of the instance in the test set

single_instance = X_test_suggestion[instance_index].toarray()

# Calculate SHAP values for the single instance
shap_values_single = explainer.shap_values(single_instance)

# Extract the SHAP values for the instance
shap_values_instance = shap_values_single[0] if isinstance(shap_values_single, list) else shap_values_single

shap_value_plot(shap_values_instance)